# PFL0091 OTIMIZAÇÃO - TRABALHO PARTICO DE OTIMIZAÇÃO

Nome: Filipe da Silva

Matricula:  20232900020

## 1. Problema

Vamos considerar o seguinte cenário: um grupo de pessoas precisa viajar para um local específico, onde se encontrarão e se hospedarão até um evento. Após o evento, eles retornarão às suas cidades de origem. Suponhamos que cada pessoa deste grupo viaje de avião, partindo de diferentes lugares do Brasil, com o destino sendo o Aeroporto de Guarulhos. Eles aguardarão a chegada do último membro do grupo para então pegarem um transporte juntos até o evento em São Paulo. Após o término do evento, eles voltarão ao Aeroporto de Guarulhos, de onde cada um seguirá para sua cidade de origem.

Agora, imaginemos que temos a responsabilidade de organizar todas as viagens, visando minimizar o tempo de espera de cada pessoa no aeroporto (afinal, pagaremos um valor por hora de espera para despesas pessoais) e buscando os preços mais baixos em cada passagem.

## 2. Motivação

Como observamos, este é um problema com múltiplas soluções, onde podemos apenas ajustar os parâmetros de forma visual, sem garantia de obter o melhor valor possível. Além disso, podemos enfrentar complicações adicionais, como a necessidade de pagar uma diária extra caso o retorno passe de um determinado horário ou a falta de transporte disponível para o retorno.

É evidente que a modelagem matemática desses dados para criar um sistema não será simples. Qualquer alteração exigirá um esforço significativo para desenvolver um novo modelo, tornando a gestão do sistema mais desafiadora e complexa.

## 3. Justificar o método de otimização escolhido para resolver o problema proposto

Decidiu-se utilizar Algoritimos Geneticos para solução, uma vez que temos um problema complexo e com múltiplas variáveis, temos que tentar buscar uma solução ótima, ou quase otima, para um problema de multiplas dimensões, como custos de viagem, tempo de espera, horários de voo e disponibilidade de transporte de forma a otimizar todas essas dimensões simultaneamente. 

Outra vantagem é que os Algoritimos Genticos possuem uma grande adaptabilidade, pois podem lidar bem com mudanças nas condições ou restrições do problema. Isso os torna adequados para situações onde as informações podem ser dinâmicas ou imprecisas. Além de ser capaz de explorar o espaço de soluções de forma abrangente, permitindo encontrar soluções que podem não ser facilmente identificadas por métodos tradicionais de otimização.

Em resumo, algoritmos genéticos são uma ferramenta poderosa e versátil para resolver problemas de otimização complexos, como os problemas de viagem, oferecendo uma abordagem adaptativa e eficiente para encontrar soluções ótimas ou quase ótimas em um espaço de soluções vasto e dinâmico.

## 4. Resolução do problema – algoritmo

Primeiramente iremos fazer a importação das bibliotecas e criamos a lista das pessoas que irão viajar com a sua origem. Após iremos importa os dados referente a origem e distino do voo, com o horário de partida e chegada, e o valor da passagem area.

In [13]:
import time
import random

peoples = [('Amanda', 'CWB'),
           ('Pedro', 'GIG'),
           ('Marcos', 'POA'),
           ('Priscila', 'FLN'),
           ('Jessica', 'CNF'),
           ('Paulo', 'GYN')]

dest = 'GRU'

flights = {}
for row in open('flights.txt'):
    #print(linha)
    src, dest, departure, arrival, price = row.split(',')
    flights.setdefault((src, dest), [])
    flights[(src, dest)].append((departure, arrival, int(price)))

Iremos criar um conjunto de funções que serviram de auxilio, sendo uma para transformar o horas em minutos e a outra para imprimir uma agenda de voo.

In [14]:
def print_schedule(sched):
    id_flight = -1
    for i in range(len(sched) // 2):
        name = peoples[i][0] 
        src = peoples[i][1]
        id_flight += 1
        going = flights[(src, dest)][sched[id_flight]]
        id_flight += 1
        ret = flights[(dest, src)][sched[id_flight]]
        print('%10s%10s %5s-%5s R$%3s %5s-%5s R$%3s' % (name, src, going[0], going[1], going[2],
                                                       ret[0], ret[1], ret[2]))

def get_min(hour):
    x = time.strptime(hour, '%H:%M')
    return x[3] * 60 + x[4]



É necessário ter o dominio do problema, no caso temos, do arquivo importado, 10 voos por dia, sendo que cada pessoa tem que pegar 2 deles.

In [15]:
domain = [(0,9)] * (len(peoples) * 2)

Agora iremos definir a função custo, que irá levar em consideração, o tempo de partida e de chegada, mais o valor do custo da passagem. Além do tempo de espera. Além de ter um penalizador pelo tempo de espera.

In [16]:
def func_cost(solve):
    total_price = 0
    last_arrival = 0
    first_departure = 1439
    
    id_flight = -1
    for i in range(len(solve) // 2):
        src = peoples[i][1]
        id_flight += 1
        going = flights[(src, dest)][solve[id_flight]]
        id_flight += 1
        ret = flights[(dest, src)][solve[id_flight]]
        
        total_price += going[2]
        total_price += ret[2]
        
        if last_arrival < get_min(going[1]):
            last_arrival = get_min(going[1])
            
        if first_departure > get_min(ret[0]):
            first_departure = get_min(ret[0])
         
    total_wait = 0
    id_flight = -1
    for i in range(len(solve) // 2):
        src = peoples[i][1]
        id_flight += 1
        going = flights[(src, dest)][solve[id_flight]]
        id_flight += 1
        ret = flights[(dest, src)][solve[id_flight]]
        
        total_wait += last_arrival - get_min(going[1])
        total_wait += get_min(ret[0]) - first_departure
        
    if last_arrival > first_departure:
        total_price += 50
        
    return total_price + total_wait



A seguir temos os 3 metodos utilizados para ter o Algoritmo genetico.

In [12]:
def mutation(domain, step, solution):
    i = random.randint(0, len(domain) - 1)
    mutante = solution
    
    if random.random() < 0.5:
        if solution[i] != domain[i][0]:
            mutante = solution[0:i] + [solution[i] - step] + solution[i + 1:]
    else:
        if solution[i] != domain[i][1]:
            mutante = solution[0:i] + [solution[i] + step] + solution[i + 1:]
    
    return mutante

def crossing(domain, solution1, solution2):
    i = random.randint(1, len(domain) - 2)
    return solution1[0:i] + solution2[i:]

def genetic_algorithm(domain, func_cost, tam_population = 100, step = 1,
             prob_mutation = 0.2, elitism = 0.2, num_generations = 500):
    
    population = []
    for i in range(tam_population):
        solution = [random.randint(domain[i][0], domain[i][1]) for i in range(len(domain))]
        population.append(solution)
    
    num_elitism = int(elitism * tam_population)
    
    for i in range(num_generations):
        costs = [(func_cost(individual), individual) for individual in population]
        costs.sort()
        ordained_individuals = [individual for (cost, individual) in costs]
        
        population = ordained_individuals[0:num_elitism]
        
        while len(population) < tam_population:
            if random.random() < prob_mutation:
                m = random.randint(0, num_elitism)
                population.append(mutation(domain, step, ordained_individuals[m]))
            else:
                c1 = random.randint(0, num_elitism)
                c2 = random.randint(0, num_elitism)
                population.append(crossing(domain, ordained_individuals[c1], 
                                            ordained_individuals[c2]))
                
    return costs[0][1]

solution_genetic_algorithm = genetic_algorithm(domain, func_cost)
cost_genetic_algorithm = func_cost(solution_genetic_algorithm)

print(cost_genetic_algorithm)

2633


In [ ]:
print_schedule(solution_genetic_algorithm)

## 5. Referencias

* https://www.feis.unesp.br/Home/departamentos/engenhariaeletrica/pos-graduacao/234-dissertacao_erico.pdf
* https://www.youtube.com/watch?v=_GuJE1AjAZ4
* https://www.ifsc.edu.br/documents/30701/523474/livro_otimizacao_parametrica_com_computacao_evolutiva.pdf/6ad9650f-9bfb-4592-a8ea-9ad8934ba085